# **CS372 Assignment 1: KNN & NB**

### **Group # : Name**

# **Summary**
Fill out the summary of this assignment here after you complete the assignment

## **Question 1**

- Download the Iris Flower data from course Moodle and use it for Question 1.
- **[2 points]** In this Question split your data 80/20. Train the model on the 80% fraction and then evaluate the accuracy on the 20% fraction. (use ['train_test_split'](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split 'train_test_split')).
    - Make sure that we preserve the proportion of each class in the splits.

#### Import Libraries

In [24]:
# import libraries in this cell
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

#### Read Iris Flower Dataset

In [25]:
# read iris Flower data
ds = pd.read_csv("Iris_Assignment 1.csv", header=0, index_col=0)

### **[16 points]** Exploratory Data Analysis
1. **[5 points]** Check for missing data and data duplication. Upon detection,
    - **[2.5 points]** show records with missing tuples and explain clearly how you dealt with them. Justify your choice.
    - **[2.5 points]** Show duplicated records and explain clearly how you dealt with them. Justify your choice.
2. **[5 points]** Draw Box Plot to spot outliers.
    - **[2 points]** Explain how [Box Plot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html) consider outliers in the dataset.
    - **[3 points]** Analyze and comment on the plot.
3. **[2 points]** Plot [Scatter Matrix](https://seaborn.pydata.org/examples/scatterplot_matrix.html) for all features. Analyze and comment on the plot.
4. **[4 points]** Plot [Heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) correlation for all features.
    - **[2 points]** Clearly indicate positively and negatively correlated features.
    - **[2 points]** Analyze and comment on the plot.

#### **1.** Check Missing Data

In [26]:
# find missing data values
for i in range(len(ds)):
    for j in ds.columns:
        if pd.isnull(ds.loc[ds.index[i],j]):
            print(f"ID: '{ds.index[i]}' Column: '{j}' is blank")
            # This will print all the IDs with missing data

ID: '2' Column: 'PetalLengthCm' is blank
ID: '20' Column: 'SepalLengthCm' is blank


#### Handle Missing Data

In [27]:
# remove the missing data
ds.dropna(inplace=True)
print("\n--------- The above data with missing fields has been removed ---------\n")
# I removed the data because the columns is data sensitive
# if I filled it with other numbers it may affect the final result


--------- The above data with missing fields has been removed ---------


#### **Explain how you dealt with missing data**

Exmplain how you dealt them, why, and justify your choice here.

#### Check Duplicate Data

In [28]:
# find duplicate records
duplicates = ds[ds.duplicated(keep=False)]
print(duplicates, "\n--------- The above duplicated data has been removed ---------\n")
# I removed the data as duplicate data may cause over-fitting

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm         Species
Id                                                                           
10             4.9           3.1            1.5           0.1     Iris-setosa
35             4.9           3.1            1.5           0.1     Iris-setosa
38             4.9           3.1            1.5           0.1     Iris-setosa
102            5.8           2.7            5.1           1.9  Iris-virginica
143            5.8           2.7            5.1           1.9  Iris-virginica 
--------- The above duplicated data has been removed ---------


#### Handle Duplicate Data

In [29]:
# remove the duplicated data
ds.drop_duplicates(inplace=True)

#### **Explain how you dealt with duplicates**

Exmplain how you dealt them, why, and justify your choice here.

#### **2.** Draw Box Plot

#### **Explain how Box Plot consider outliers in the dataset**

Add comments here

#### **Analyze and comment on the box plot**

Add comments here

#### **3.** Plot Scatter Matrix

#### **Analyze and comment on the scatter matrix plot**

Add comments here

#### **4.** Plot Heatmap Correlation

#### **Analyze and comment on the heatmap correlation plot**

Add comments here

### KNN **[15 points]**

1. **[6 points]** Implement KNN using sklearn library (use [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)).
    - **[4 points]** Experiment with different distance metrics, namely, Euclidean metric and Manhattan distance. Comment on your findings.
    - **[2 points]** Use [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to put all attributes on the same scale.
2. **[5 points]** Choose the best number of neighbors using [5-fold cross validation](https://scikit-learn.org/stable/modules/cross_validation.html) (from built-in) approach we studied in the class.
3. **[2 points]** K value vs. [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html): draw a 2D plot to show the average accuracy of KNN classifier vs different number of k’s. Analyze your result and comment on your plot.
    - Evaluate your model on the test data using Accuracy based on best K found in (2) & (3). Comment on the result and print misclassified flowers in table format where you show the true label in one column and the predicted table in another column.

#### **1.** StandardScaler: put all attributes on the same scale

#### Split your data 80/20

In [30]:
# shuffle and split

#### Implement KNN using Euclidean metric and Manhattan distance

#### **Analyze and comment on your finding**

Analyze KNN with different metrics regarding accuracy, time, etc.

#### **2.** Choose the Best Number of Neighbors (5-Fold Cross Validation)

#### **3.** Draw Plot : K value vs Accuracy

#### Evaluate your model on the test data using Accuracy based on K found in (2) & (3)

#### Print misclassified flowers

#### **Analyze and comment on the result**

Evaluate your model on the test data using Accuracy based on best K found in (2) & (3). Comment on the result and print misclassified flowers in table format where you show the true label in one column and the predicted table in another column.

### Naive Bayes **[9 points]**
1. **[5 points]** You need to implement Naïve Bayes using the [built-in library](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) and use it to predict the Species of iris in the test dataset.
2. **[4 points]** Use [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) to assess the performance of your classifier.
    - **[2 points]** Print misclassified flowers in table format where you show the true label in one column and the predicted table in another column.

#### **1.** Implement Naive Bayes

#### **2.** Assess the Performance (Accuracy)

#### Print misclassified flowers

## **Question 2**

**Problem description**: Predict the onset of diabetes based on diagnostic measures.
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.
Some information about the data :
- Number of pregnancies.
- Body Mass Index (BMI)
- Insulin level.
- Age.
- Glucose.
- Blood Pressure.
- Diabetes Pedigree Function.
- Outcome (Target Feature).
  
1. Download the Pima Indians Diabetes Database data from course Moodle.
2. Split your data 80/20. Train the model on the 80% fraction and then evaluate the accuracy on the 20% fraction.

### **[16 points]** Exploratory Data Analysis
3. **[3 points]** Check for missing data and data duplication. Upon detection, show and explain clearly how you dealt with them. Justify your choice.
4. **[2 points]** Draw Box Plot to spot outliers. Analyze and comment on the plot.
5. **[2 points]** Plot scatter matrix for all features. Analyze and comment on the plot.
6. **[3 points]** Plot Heatmap correlation for all features. Analyze and comment on the plot.

#### **3.** Check Missing Data

In [31]:
# check missing data and show the records

#### Handle Missing Data

In [32]:
# handle the missing data

#### Check Duplicate Data

#### Handle Duplicate Data

#### **Explain how you dealt with missing data and duplicates**

Exmplain how you dealt them, why, and justify your choice here.

#### **4.** Draw Box Plot

#### **Analyze and comment on the box plot**

Add comments here

#### **5.** Plot Scatter Matrix

#### **Analyze and comment on the scatter matrix plot**

Add comments here

#### **6.** Plot Heatmap Correlation

#### **Analyze and comment on the heatmap correlation plot**

Add comments here

#### Split your data 80/20

In [33]:
# shuffle and split

### **Question 2 Part 1**
### **[20 points]** Implement Gaussian Naïve Bayes from scratch.
- **[10 points]** Make sure to use functions in writing your codes.
- **[5 points]** Underflow prevention.
- **[5 points]** Zero variance.

#### Implement Function for Gaussian Naive Bayes

#### Evaluate your own Gaussian Naive Bayes

#### Accuracy

### **[20 points] Question 2 Part 2**
8. **[12 points]** You need to implement KNN from scratch:
- **[5 points]** Implement Euclidean Distance from scratch (please, don’t use built-in library).
- **[2 points]** Use [Min-Max](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to normalize the dataset.
9. **[9 points]** Tune the number of nearest neighbors k
- What is the optimal value of k? To figure it out:
    - **[5 points]** Implement N-fold-cross validation from scratch:
        - Train KNN classifier using 5-fold-cross validation using various values of k.
            - Explain/ justify the choice of the range of K values.
            - Choose k that obtained the highest accuracy.
    - Next, evaluate the classifier using the optimal value of K that we found using 5-fold cross validation on the test set (20% that we did not use in the 5-fold cross validation) and obtain the final results.
- **[2 points]** In case of tie, KNN algorithm prefers the neighbor with closer distance to the query.
- **[2 points]** K value vs. Accuracy: draw a 2D plot to show the accuracy of KNN classifier vs different number of k’s.

#### **8.** Function for KNN (from scratch) 
- Metric: Euclidean Distance
- Normalize: [Min-Max](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
- In case of tie, KNN algorithm prefers the neighbor with closer distance to the query

#### **9.** Tune the Number of Nearest Neighbors K

#### Implement function for N-Fold-Cross Validation (from scratch)

#### Train KNN Classifier using 5-Fold-Cross Validation with Various Values of K

#### Accuracy of KNN Classifier with Various K

In [34]:
#Calculate accuracy

#### **Analyze and comment on choice of the range of K values**

Explain/justify the choice of the range of K values. Choose K that obtained the highest accuracy

#### Evaluate KNN Classifier with the Best K

In [35]:
#final result

#### Print missclassified data

#### **Analyze and comment on the result**

Evaluate your model on the test data using Accuracy based on best K found in (9). Comment on the result and print misclassified data in table format where you show the true label in one column and the predicted table in another column.

### **[8 points]** Model Evaluation
10. **[8 points]** Compare, comment, and analyze the result of all the classifiers you built thus far Naïve Bayes & KNN using built-in library (you implemented in Question 1) with Naïve Bayes & KNN from scratch implementation in Question 2 for Pima Indians Diabetes Database data.
- **[6 points]** To compare the performance of your classifier use:
    - **[2 points]** execution time.
    - **[2 points]** Accuracy
    - **[2 points]** [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay).

#### Implement Naive Bayes(built-in library) for Pima Indians Diabetes Databaset data

#### Compare built-in Naive Bayes and your Naive Bayes regarding execution time, accuracy, and confusion matrix

#### **Comment on the comparison (built-in Naive Bayes vs from scratch)**

Add comment here

#### Implement KNN(built-in library) for Pima Indians Diabetes Databaset data

#### Compare built-in KNN and KNN from scratch regarding execution time, accuracy, and confusion matrix

#### **Comment on the comparison (built-in Naive Bayes vs from scratch)**

Add comment here

#### Compare Naive Bayes and KNN (both from scratch) regarding execution time, accuracy, and confusion matrix

#### **Compare, comment, and analyze the result of ALL classifiers**

Add comment here